In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('D:/学习/集思项目/final project/train.csv')
X = pd.read_csv(r'D:/学习/集思项目/final project/tfidf2.csv',header = None)

In [3]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,34634,34635,34636,34637,34638,34639,34640,34641,34642,34643
0,4.482953,30.570149,18.022587,6.971426,5.800844,8.37292,4.12232,2.206366,12.935958,2.680922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,2.206366,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = np.array(X)
Y = data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']]
Y = Y.head(10000)
Y = np.array(Y)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [6]:
X_sum = np.row_stack((X_train, X_test))
Y_sum = np.row_stack((y_train, y_test))

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

X_sparse = sparse.csr_matrix(X_sum)

In [8]:
X_sparse

<10000x34644 sparse matrix of type '<class 'numpy.float64'>'
	with 686140 stored elements in Compressed Sparse Row format>

In [9]:
similarities = cosine_similarity(X_sparse)

In [10]:
similarities

array([[1.00000000e+00, 8.59709375e-03, 8.03943867e-03, ...,
        2.83923161e-03, 4.74126284e-03, 5.39335239e-02],
       [8.59709375e-03, 1.00000000e+00, 1.82865448e-03, ...,
        2.01976232e-03, 3.98288973e-03, 3.42897602e-03],
       [8.03943867e-03, 1.82865448e-03, 1.00000000e+00, ...,
        3.18815544e-03, 1.06452928e-02, 3.92124789e-03],
       ...,
       [2.83923161e-03, 2.01976232e-03, 3.18815544e-03, ...,
        1.00000000e+00, 8.43437042e-03, 9.41189954e-04],
       [4.74126284e-03, 3.98288973e-03, 1.06452928e-02, ...,
        8.43437042e-03, 1.00000000e+00, 1.18537058e-02],
       [5.39335239e-02, 3.42897602e-03, 3.92124789e-03, ...,
        9.41189954e-04, 1.18537058e-02, 1.00000000e+00]])

In [11]:
def fit_ML_KNN(k, X, Y, similarities):
    
    s = 1
    m = len(X)
    P1 = np.zeros((6,))
    Pe1 = np.zeros((6, k + 1))
    P0 = np.zeros((6,))
    Pe0 = np.zeros((6, k + 1))
    ind = np.argsort(-similarities)
    
    for i in range(6):
        y = 0
        for j in range(m):
            if Y[j][i] == 1:
                y = y + 1
        P1[i] = (s + y) / (s * 2 + m)
        P0[i] = 1 - P1[i]
    
    for i in range(6):
        c1 = np.zeros(k + 1)
        c0 = np.zeros(k + 1)
        for j in range(m):
            temp = 0
            for kk in range(1,k + 1):
                if Y[ind[j][kk]][i] == 1:
                    temp = temp + 1
            if Y[j][i] == 1:
                c1[temp] = c1[temp] + 1
            else:
                c0[temp] = c0[temp] + 1
        
        for l in range(k + 1):
            Pe1[i][l] = (s + c1[l]) / (s * (k + 1) + c1.sum())
            Pe0[i][l] = (s + c0[l]) / (s * (k + 1) + c0.sum())
            
    return P1, P0, Pe1, Pe0

In [12]:
def predict_ML_KNN(k, P1, P0, Pe1, Pe0, test_Data, Y, similarities):
    
    rtl = np.zeros((test_Data.shape[0], 6))
    m = test_Data.shape[0]
    predict_labels = np.zeros((m, 6))
    ind = np.argsort(-similarities)
    
    for i in range(m):
        for j in range(6):
            temp = 0
            y1 = 0
            y0 = 0
            f = 1
            count = 0
            while count < k:
                if ind[i + 8000][f] >= 8000:
                    f = f + 1
                    continue
                else:
                    count = count + 1
                    if Y[ind[i + 8000][f]][j] == 1:
                        temp = temp + 1
                    f = f + 1
            y1 = P1[j] * Pe1[j][temp]
            y0 = P0[j] * Pe0[j][temp]
            rtl[i][j] = P1[j] * Pe1[j][temp] / (P1[j] * Pe1[j][temp] + P0[j] * Pe0[j][temp])
            if y1 > y0:
                predict_labels[i][j] = 1
            else:
                predict_labels[i][j] = 0
    
    return predict_labels, rtl

In [13]:
def hloss(predict_label, true_label):
  #预测集行数等于验证集大小，预测集列数等于验证集label大小
    data_num = predict_label.shape[0]
    label_num = predict_label.shape[1]
    hloss = 0
    for i in range(data_num):
        bias = 0
        for j in range(label_num):
            if predict_label[i,j] != true_label[i,j]:
                bias += 1
        hloss += bias/label_num
    hloss = hloss/data_num
    return hloss



def one_error(predict_rt, true_label): 
    
    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    one_error_num = 0
    for i in range(data_num):
        rtl = list(predict_rt[i])
        max_index = rtl.index(max(rtl))
        if true_label[i,max_index] != 1:
            one_error_num += 1
    
    return one_error_num/data_num

def sort_both(list_rtl, list_target):
    for j in range(len(list_rtl)-1):
        for k in range(j,len(list_rtl)):
            if list_rtl[j] <= list_rtl[k]:
                temp1 =list_rtl[j]
                temp2 =list_target[j]
                list_rtl[j] = list_rtl[k]
                list_target[j] = list_target[k]
                list_rtl[k] = temp1 
                list_target[k] = temp2
    
    return list_rtl,list_target

def coverage(predict_rt, true_label):
    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    coverage1 = 0
    for i in range(data_num):
        max_rank = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)

        for j in range(len(rtl)):
            if target[j] == 1:
                max_rank = j+1
        coverage1 += max_rank
    
    return (coverage1/data_num-1)


def rloss(predict_rt, true_label):
    #import pdb; pdb.set_trace()

    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    r_loss = 0
    for i in range(data_num):
        index_1 = []
        index_0 = []
        rank_loss = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)
        for j in range(label_num):
            if target[j] == 1:
                index_1.append(j)
            else:
                index_0.append(j)
      
        m = len(index_1)
        n = len(index_0)
        for j in range(m):
            for k in range(n):
                if index_0[k] <= index_1[j]:
                    rank_loss += 1
      
        r_loss += rank_loss/(m*n)
    return r_loss/data_num
  

def avgprec(predict_rt, true_label):

    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    avg_prec = 0
    for i in range(data_num):
        index_1 = []
        temp_prec = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)
        for j in range(label_num):
            if target[j] == 1:
                index_1.append(j)
        for k in range(len(index_1)):
            temp_prec += (k+1)/(index_1[k]+1)
      
        avg_prec += temp_prec/len(index_1)
    
    return avg_prec/data_num

In [14]:
X_train_sparse = sparse.csr_matrix(X_train)
X_train_similarities = cosine_similarity(X_train_sparse)
true_labels = Y[8000:][:]
kk = 20
Hamming_Loss = np.zeros(kk)
One_Error = np.zeros(kk)
Coverage = np.zeros(kk)
Ranking_Loss = np.zeros(kk)
Average_Precision = np.zeros(kk)

for k in range(kk):
    P1, P0, Pe1, Pe0 = fit_ML_KNN(k, X_train, y_train, X_train_similarities)
    predict_labels, predict_rtl = predict_ML_KNN(k, P1, P0, Pe1, Pe0, X_test, Y_sum, similarities)
    Hamming_Loss[k] = hloss(predict_labels, true_labels)
    One_Error[k] = one_error(predict_rtl, true_labels)
    Coverage[k] = coverage(predict_rtl, true_labels)
    Ranking_Loss[k] = rloss(predict_rtl, true_labels)
    Average_Precision[k] = avgprec(predict_rtl, true_labels)
    print('k = ',k)
    print('Hamming Loss is',Hamming_Loss[k])
    print('One Error is',One_Error[k])
    print('Coverage is',Coverage[k])
    print('Ranking Loss is',Ranking_Loss[k])
    print('Average Precision is',Average_Precision[k])
    print('\n')

k =  0
Hamming Loss is 0.20583333333333587
One Error is 0.6
Coverage is 1.5394999999999999
Ranking Loss is 0.24941805555555305
Average Precision is 0.6103527777777791


k =  1
Hamming Loss is 0.28116666666666545
One Error is 0.6775
Coverage is 1.7320000000000002
Ranking Loss is 0.2956000000000007
Average Precision is 0.5646666666666689


k =  2
Hamming Loss is 0.2558333333333324
One Error is 0.667
Coverage is 1.7405
Ranking Loss is 0.29728888888888905
Average Precision is 0.5670861111111141


k =  3
Hamming Loss is 0.2764999999999991
One Error is 0.6725
Coverage is 1.7275
Ranking Loss is 0.2937513888888883
Average Precision is 0.5671513888888914


k =  4
Hamming Loss is 0.27049999999999885
One Error is 0.6825
Coverage is 1.7599999999999998
Ranking Loss is 0.3019805555555559
Average Precision is 0.5600472222222249


k =  5
Hamming Loss is 0.2772499999999992
One Error is 0.688
Coverage is 1.7715
Ranking Loss is 0.3034486111111106
Average Precision is 0.556400000000003


k =  6
Hamming Lo

In [15]:
import matplotlib.pyplot as plt

X = np.arange(kk)
plt.plot(X, Hamming_Loss)
plt.plot(X, One_Error)
plt.plot(X, Coverage)
plt.plot(X, Ranking_Loss)
plt.plot(X, Average_Precision)

In [16]:
Hamming_Loss

array([0.20583333, 0.28116667, 0.25583333, 0.2765    , 0.2705    ,
       0.27725   , 0.27483333, 0.27733333, 0.27383333, 0.278     ,
       0.27691667, 0.2745    , 0.27766667, 0.27716667, 0.27808333,
       0.27658333, 0.27966667, 0.27516667, 0.27983333, 0.27808333])

In [17]:
One_Error

array([0.6   , 0.6775, 0.667 , 0.6725, 0.6825, 0.688 , 0.694 , 0.696 ,
       0.692 , 0.692 , 0.6935, 0.6915, 0.6885, 0.689 , 0.69  , 0.691 ,
       0.685 , 0.69  , 0.6905, 0.6895])

In [18]:
Coverage

array([1.5395, 1.732 , 1.7405, 1.7275, 1.76  , 1.7715, 1.7835, 1.7905,
       1.8115, 1.8165, 1.8305, 1.8355, 1.841 , 1.8435, 1.849 , 1.848 ,
       1.8475, 1.8485, 1.831 , 1.8365])

In [19]:
Ranking_Loss

array([0.24941806, 0.2956    , 0.29728889, 0.29375139, 0.30198056,
       0.30344861, 0.306975  , 0.30709583, 0.31094028, 0.3121375 ,
       0.31555278, 0.31655833, 0.31775   , 0.31702083, 0.31909306,
       0.31903056, 0.31924861, 0.31986667, 0.31664722, 0.31769861])

In [20]:
Average_Precision

array([0.61035278, 0.56466667, 0.56708611, 0.56715139, 0.56004722,
       0.5564    , 0.55142083, 0.55118333, 0.5494125 , 0.55033889,
       0.54838611, 0.54828611, 0.54902639, 0.54887778, 0.54759306,
       0.54736806, 0.5492375 , 0.54795278, 0.54961528, 0.54957917])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

date_palatte = [(0.20651240887022015, 0.3755393698154809, 0.5856904756194529),
 (0.3720866446574984, 0.6378633419526029, 0.5550368905837924),
 (0.15908978777909805, 0.2627191708800849, 0.2704304083743111),
 (0.18673086084284554, 0.40157226483269637, 0.4175589767109701)]

data = pd.DataFrame(columns = ['Hamming Loss','One Error','Coverage','Ranking Loss','Average Precision'])
data['Hamming Loss'] = Hamming_Loss
data['One Error'] = One_Error
data['Coverage'] = Coverage
data['Ranking Loss'] = Ranking_Loss
data['Average Precision'] = Average_Precision
data.to_excel